In [ ]:
import numpy as np
import ioh

In [ ]:
ioh.ProblemClass.GRAPH.problems

In [ ]:
max_cut = ioh.get_problem(2000, problem_class=ioh.ProblemClass.GRAPH)
max_coverage = ioh.get_problem(2100, problem_class=ioh.ProblemClass.GRAPH)

In [ ]:
print(max_cut.meta_data)
print(max_coverage.meta_data)

In [ ]:
nb_it = 10000

In [ ]:
x_mcov = cut = np.random.randint(0,2,max_coverage.meta_data.n_variables)
val = max_coverage(x_mcov)
# help(max_coverage.optimum)
for ind in ioh.ProblemClass.GRAPH.problems:
    p = ioh.get_problem(ind, problem_class=ioh.ProblemClass.GRAPH)
    if np.any(p.optimum.x):
        print(p.meta_data)

# 1/2-approximation for MaxCut

Proof in Bertrand Meyer's Advanced Algorithms lecture

In [ ]:
max_cut.reset()

In [ ]:
cut = np.random.randint(0,2,max_cut.meta_data.n_variables)
max_cut(cut)
print(max_cut.state)

# RLS

In [ ]:
import numpy as np

class RandomLocalSearch:
    'Random local search algorithm'
    def __init__(self, n: int):
        self.n: int = n
        
    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        'For each it flip 1 bit chosen uar and keep the best of both'
        
        x = np.random.randint(0,2,problem.meta_data.n_variables)
        val = problem(x)
        for _ in range(self.n):
            index_to_flip = np.random.randint(0,problem.meta_data.n_variables)
            y = x
            y[index_to_flip] = 1 - y[index_to_flip]
            val_y = problem(y)
            if val_y > val:
                x = y
                val = val_y

In [511]:
max_coverage.reset()
r = RandomLocalSearch(nb_it)
r(max_coverage)
print(max_coverage.state.evaluations, max_coverage.state.y_unconstrained, max_coverage.state.y_unconstrained_best)

10001 -221.0 -185.0


In [513]:
max_cut.reset()
r = RandomLocalSearch(nb_it)
r(max_cut)
print(max_cut.state.evaluations, max_cut.state.y_unconstrained, max_cut.state.y_unconstrained_best)

10001 9527.0 9749.0


# (1+1) EA 

In [ ]:
class EA_1p1:
    '(1+1)-EA'
    def __init__(self, n: int, p : float):
        self.n: int = n
        self.p: float = p
        
    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        x = np.random.randint(0,2,problem.meta_data.n_variables)
        val = problem(x)
        for _ in range(self.n):
            y = x.copy()
            for ind in range(len(x)):
                y[ind] = 1 - y[ind] if np.random.binomial(1,self.p) else y[ind]
            val_y = problem(y)
            if val_y > val:
                x = y
                val = val_y

In [ ]:
max_coverage.reset()
ea_11 = EA_1p1(nb_it, 1/max_coverage.meta_data.n_variables)
ea_11(max_coverage)
print(max_cut.state.evaluations, max_cut.state.y_unconstrained, max_cut.state.y_unconstrained_best)

In [ ]:
max_cut.reset()
ea_11 = EA_1p1(nb_it, np.log(max_cut.meta_data.n_variables)/max_cut.meta_data.n_variables)
ea_11(max_cut)
print(max_cut.state)

# (1+($\lambda, \lambda$)) EA 

In [ ]:
max_cut.reset()

In [ ]:
import random

class EA_1plamlam:
    '(1+(lam,lam))-EA'
    def __init__(self, n: int, lam : int, p : float, c : float):
        self.n: int = n
        self.lam: int = lam
        self.p: float = p
        self.c: float = c
        
    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        length_x = problem.meta_data.n_variables
        x = np.random.randint(0,2,length_x)
        val = problem(x)
        for _ in range(self.n):
            L = np.random.binomial(length_x, self.p)
            xp = None
            best_val = None # we allow for worse values than x
            for _ in range(self.lam):
                bits_to_flip = random.sample(range(length_x), L)
                new_xp = x.copy()
                for i in bits_to_flip:
                    new_xp[i] = 1 - new_xp[i] if np.random.binomial(1, self.p) else new_xp[i]
            val_new_xp = problem(new_xp)
            if best_val is None or val_new_xp > best_val:
                xp = new_xp
                best_val = val_new_xp 
            y = x.copy()
            best_val = None
            for _ in range(self.lam):
                new_y = x.copy()
                for i in range(length_x):
                    if np.random.binomial(1,self.c):
                        new_y[i] = xp[i]
                val_new_y = problem(new_y)
                if best_val is None or val_new_y > best_val:
                    y = new_y
                    best_val = val_new_y
            if best_val > val:
                x = y
                val = best_val

In [430]:
import math
max_coverage.reset()
dim_pb = max_coverage.meta_data.n_variables
ea_1lamlam = EA_1plamlam(math.ceil(nb_it / (10)), 10, 1/dim_pb, 1/dim_pb)
ea_1lamlam(max_coverage)
print(max_coverage.state.evaluations, max_coverage.state.y_unconstrained, max_coverage.state.y_unconstrained_best)

11001 -198.0 -198.0


In [ ]:
import math
max_cut.reset()
dim_pb = max_cut.meta_data.n_variables
ea_1lamlam = EA_1plamlam(math.ceil(nb_it / (10)), 10, 1/dim_pb, 1/dim_pb)
ea_1lamlam(max_cut)
print(max_cut.state.evaluations, max_cut.state.y_unconstrained, max_cut.state.y_unconstrained_best)

# First idea 
as dealing with submodular functions, we may want to select the arguments that make it grow "the most at first" --> as when adding more elements we face diminishing returns
so from a solution x -> we build lam xs turning off L bits of uar and keeping the best -> we then build lam ys by flipping on L' bits of the best xs uniformly at random and keeping the best y --> then we turn on each bit present in y and not in x

problem: solution x keeps growing?
how to stop that?

In [ ]:
class OffAndOn:
    def __init__(self, n: int, lam : int, p_off : float, p_on : float, c : float):
        self.n: int = n
        self.p_off: int = p_off
        self.p_on: int = p_on
        self.lam: float = lam
        self.c: bool = c
        
        
    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        x = np.zeros(problem.meta_data.n_variables, dtype=int)
        val = problem(x)
        x_ = np.random.randint(0,2,problem.meta_data.n_variables)
        val_ = problem(x)
        if val_ > val:
            x = x_
        for _ in range(self.n):
            L = np.random.binomial(problem.meta_data.n_variables, self.p_off)
            xp = None
            best_val = None 
            for _ in range(self.lam):
                # mutation 1 : we turn off L bits 
                bits_to_flip = random.sample(range(problem.meta_data.n_variables), L)
                new_xp = x.copy()
                for i in bits_to_flip:
                    new_xp[i] = 0
            val_new_xp = problem(new_xp)
            if best_val is None or val_new_xp > best_val:
                xp = new_xp
                best_val = val_new_xp 
            L = np.random.binomial(problem.meta_data.n_variables, self.p_on)
            y = xp.copy()
            best_val = None
            for _ in range(self.lam):
                # mutation 2 : we turn on L bits
                new_y = x.copy()
                bits_to_flip = random.sample(range(problem.meta_data.n_variables), L)
                for i in bits_to_flip:
                    new_y[i] = 1
                val_new_y = problem(new_y)
                if best_val is None or val_new_y > best_val:
                    y = new_y
                    best_val = val_new_y
            # mix:
            # if bit in y and not in x then I turn it on 
            mix = [0] * problem.meta_data.n_variables
            for i in range(problem.meta_data.n_variables):
                pc = np.random.binomial(1, self.c)
                if y[i] and not x[i]:
                    mix[i] = 1 if not pc else 0
                else:
                    mix[i] = x[i] if not pc else y[i]
            val_new = problem(mix)
            if val_new > val:
                x = mix
                val = best_val
            

In [518]:
import math
max_cut.reset()
dim_pb = max_cut.meta_data.n_variables
lam = 10
offandon = OffAndOn(math.ceil(nb_it / (lam + 1)), lam, lam/dim_pb, 1/dim_pb, False, lam/dim_pb)
offandon(max_cut)
print(max_cut.state.evaluations, max_cut.state.y_unconstrained_best)

10921 9971.0


In [515]:
import math
max_coverage.reset()
dim_pb = max_coverage.meta_data.n_variables
lam = 10
offandon = OffAndOn(math.ceil(nb_it / (lam + 1)), lam, lam/dim_pb, 1/dim_pb, True, lam/dim_pb)
offandon(max_coverage)
print(max_coverage.state.evaluations, max_coverage.state.y_unconstrained_best)

10921 425.0


# Second idea:

from x:
 - generate lam children by flipping bits randomly 
 - take the lam2 best and do the intersection --> those are the "best elements to keep"
 - combine x and the intersection

In [439]:
class IntersectAndCombine:
    def __init__(self, n: int, lam: int, lam2: int, p: float):
        self.n: int = n
        self.p: int = p
        self.lam: float = lam
        self.lam2: float = lam2

    def generate_child(self, x):
        flip_mask = np.random.binomial(1, self.p, size=len(x))
        return np.where(flip_mask, 1 - x, x)

    def intersection(self, x, y):
        return np.logical_and(x, y).astype(int)

    def union(self, x, y):
        return np.logical_or(x, y).astype(int)

    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        x = np.zeros(problem.meta_data.n_variables, dtype=int)
        val = problem(x)
        x_ = np.random.randint(0, 2, problem.meta_data.n_variables)
        val_ = problem(x)
        if val_ > val:
            x = x_
            val = val_
        for _ in range(self.n):
            children = []
            for _ in range(self.lam):
                x_ = self.generate_child(x)
                children.append((problem(x_), x_))
            children.sort(key=lambda x: x[0])

            intersection = [self.intersection(children[i][1], children[i+1][1]) for i in range(0, self.lam2, 2)]
            union = np.sum(intersection, axis=0)
            union = np.where(union > 0, 1, 0)

            val_ = problem(union)
            if val_ > val:
                x = union
                val = val_

In [514]:
import math
max_cut.reset()
dim_pb = max_cut.meta_data.n_variables
lam = 10
intcomb = IntersectAndCombine(1000, lam, 4, 1/dim_pb)
intcomb(max_cut)
print(max_cut.state.evaluations, max_cut.state.y_unconstrained_best)

11002 384.0


In [509]:
import math
max_coverage.reset()
dim_pb = max_coverage.meta_data.n_variables
lam = 20
intcomb = IntersectAndCombine(500, lam, 10, 10/dim_pb)
intcomb(max_coverage)
print(max_coverage.state.evaluations, max_coverage.state.y_unconstrained_best)

10502 402.0
